# Sampling from the  sparse semi-definite positive matrix space

Here, we define a procedure to simulate a sparse semi-definite positive matrix.

We first load **matplotlib** software and enable plots embedded in the notebook (see [documentation](https://matplotlib.org/contents.html)).

In [1]:
import matplotlib
%matplotlib nbagg
from matplotlib import pyplot

Then, we load **StatisKit** software suite required for performing high-dimensional graph inference:

* The **StatisKit.LinAlg** software is necessary for solving linear algebra problems in *C++* and *Python*.
* The **StatisKit.Core** software is necessary for manipulating classical statistical tools in *C++* and *Python*.
* The **StatisKit.PGM** software is necessary for using probalistic graphical models in *C++* and *Python*.

In [2]:
from statiskit import (linalg,
                       core,
                       pgm)

Note that we need to set the seed from an OS' random number generator to have different results for each execution.

In [3]:
import os
seed = ord(os.urandom(1))
core.controls.set_seed(seed)

Let $K$ be the problem dimension.

In [4]:
K = int(os.environ.get('K', 10))

We use the mixture model for undirected graphs to simulate a graph $G = \left(\mathcal{V}, \mathcal{E}\right)$ denoted by `graph` in the code.
The mixture model is defined with $3$ states with

$$\alpha = \left(0.2, 0.1, 0.7\right),$$

and

$$
\Pi = \begin{pmatrix}
            0.8 & 0.5 & 0.1\\
            0.5 & 0.3 & 0.1\\
            0.1 & 0.1 & 0.1\\
      \end{pmatrix}.
$$

In [5]:
process = pgm.UndirectedGraphProcess("mixture",
                                     nb_vertices=K,
                                     nb_states=3,
                                     pi = linalg.Matrix([[0.8, 0.5, 0.1],
                                                         [0.5, 0.3, 0.1],
                                                         [0.1, 0.1, 0.1]]),
                                     alpha = linalg.Vector([0.1, 0.05, 0.85]))
graph = process.simulate()
graph.write_gml(os.path.join('results', 'graph.gml'))

We here represent the adjacency matrix $A$ of $G$ and $\tilde{A}$ that is a permutation of $A$ such as vertices are regrouped by their MAP assignment.

In [6]:
fig = pyplot.figure()
axes = graph.to_matrix().plot(axes = fig.add_subplot(121))
axes.set_title(r'$A$')
assigment = list(process.assignment(graph))
axes = graph.to_matrix(sort = assigment).plot(axes = fig.add_subplot(122))
axes.set_title(r'$\tilde{A}$')

<IPython.core.display.Javascript object>

To obtain a concentration matrix $\Theta$ that has the same sparsity pattern as $G$, we first simulate a covariance matrix $\Sigma$ using a [Wishart distribution](https://en.wikipedia.org/wiki/Wishart_distribution) $W_K\left(V, K\right)$ where,
$$
    \forall \left(u, v\right) \in \mathcal{V}^2,\,
                  \begin{cases}
                      v_{u,v} = 2 & \mathrm{if}\, i = j,\\
                      v_{u,v} = 1 & \mathrm{otherwise.}\\
                  \end{cases}
$$
This matrix $\Sigma$ is the covariance matrix of the `data` variable obtained by drawing $K$ independent samples from the random vector $\boldsymbol{X} \sim \mathcal{N}\left(\boldsymbol{0}, V\right)$.

In [7]:
dist = pgm.GraphicalGaussianDistribution(linalg.Vector([0] * K),
                                         linalg.Matrix([[2 if j == i else 1 for j in range(K)] for i in range(K)]))
data = dist.simulation(K)
fig = pyplot.figure()
axes = data.covariance.plot(axes = fig.add_subplot(111))
axes.set_title('$\Sigma$')
sigma = data.covariance
#print(sigma)

<IPython.core.display.Javascript object>

Hence, the concentration matrix $\Theta$ such as for all pair $\left(u, v\right)$ of disjoint vertices of $\mathcal{V}$, $\theta_{u, v} = 0$ if, and only if, $\left(u, v\right) \not\in \mathcal{E}$ is obtained by infering the parameters of a gaussian graphical model from `data` with the graph $G$.
`algo` denotes the algorithm used for infering the parameters of a gaussian graphical model.
The possible values for `algo` are:

* `'ml'` (uniquely for chordal graphs)
* `'cd'`
* `'nr'`
* `'scd'` is an optimized version of `'cd'` taking advantage of the sparsity pattern of $\Theta$ to compute its inverse.

In [8]:
%%time
mle = pgm.graphical_gaussian_estimation(algo='scd',
                                        data=data,
                                        graph=graph)

CPU times: user 6.77 ms, sys: 317 µs, total: 7.09 ms
Wall time: 7.1 ms


As illustrated above, the estimated $\widehat{\Theta}$ concentration matrix has the same sparsity pattern has $G$ and is a semi-definite positve matrix that can be used as concentration matrix for gaussian graphical models.

In [9]:
fig = pyplot.figure()
axes = graph.to_matrix().plot(axes = fig.add_subplot(131))
axes.set_title(r'$A$')
axes = mle.estimated.graph.to_matrix().plot(axes = fig.add_subplot(132))
axes.set_title(r'$\widehat{A}$')
axes = mle.estimated.theta.plot(axes = fig.add_subplot(133))
axes.set_title(r'$\widehat{\Theta}$')

<IPython.core.display.Javascript object>

We can therefore simulate a new data set that will be used for large-scale tests such as the study of the graphical lasso performance.

In [10]:
data = mle.estimated.simulation(K)
data.write_csv(os.path.join('results', 'data.csv'))

In [11]:
data

,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19
0,$2.55$,$0.36$,$0.55$,$-0.32$,$-0.50$,$-0.77$,$-0.01$,$0.77$,$1.08$,$0.30$
1,$1.48$,$1.18$,$-0.19$,$0.24$,$-0.09$,$-0.45$,$0.02$,$0.90$,$-0.11$,$-0.33$
2,$-1.95$,$1.86$,$-0.95$,$-1.04$,$0.56$,$-1.57$,$-0.14$,$0.30$,$0.38$,$-0.25$
3,$1.58$,$0.72$,$-0.34$,$-0.27$,$-0.05$,$-0.25$,$0.85$,$-0.32$,$0.73$,$0.08$
4,$-0.13$,$0.51$,$-0.20$,$1.31$,$-1.10$,$0.67$,$-0.71$,$0.96$,$-0.60$,$-0.20$
5,$-1.74$,$1.50$,$0.12$,$0.28$,$0.77$,$-1.29$,$0.91$,$-0.33$,$-0.28$,$1.05$
6,$1.32$,$-0.56$,$-0.43$,$1.63$,$-0.37$,$0.62$,$-1.37$,$-0.14$,$-1.33$,$0.93$
7,$2.18$,$0.53$,$-1.92$,$3.27$,$0.53$,$1.77$,$-0.71$,$-1.05$,$1.26$,$1.21$
8,$1.30$,$-1.00$,$-0.76$,$-2.20$,$-1.07$,$-0.51$,$-0.25$,$-0.18$,$0.45$,$0.89$
9,$1.03$,$0.17$,$0.47$,$-1.96$,$0.19$,$-1.03$,$-0.07$,$-0.39$,$0.40$,$0.29$
